In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
tf.__version__

'2.7.0'

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/New Plant Diseases Dataset/train'
valid_path = '/content/drive/MyDrive/New Plant Diseases Dataset/test'

In [ ]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87924736/87910968 [==============================] - 3s 0us/step


In [ ]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [ ]:
# useful for getting number of output classes
folders = glob('/content/drive/MyDrive/New Plant Diseases Dataset/train/*')

In [ ]:
folders

['/content/drive/MyDrive/New Plant Diseases Dataset/train/Tomato___Late_blight',
 '/content/drive/MyDrive/New Plant Diseases Dataset/train/Tomato___Early_blight',
 '/content/drive/MyDrive/New Plant Diseases Dataset/train/Tomato___Leaf_Mold',
 '/content/drive/MyDrive/New Plant Diseases Dataset/train/Tomato___healthy',
 '/content/drive/MyDrive/New Plant Diseases Dataset/train/Tomato___Tomato_mosaic_virus',
 '/content/drive/MyDrive/New Plant Diseases Dataset/train/Tomato___Septoria_leaf_spot',
 '/content/drive/MyDrive/New Plant Diseases Dataset/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 '/content/drive/MyDrive/New Plant Diseases Dataset/train/Tomato___Spider_mites Two-spotted_spider_mite',
 '/content/drive/MyDrive/New Plant Diseases Dataset/train/Tomato___Target_Spot',
 '/content/drive/MyDrive/New Plant Diseases Dataset/train/Tomato___Bacterial_spot']

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D
# our layers - you can add more if you want
x = GlobalAveragePooling2D()(inception.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                           

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='SGD',
  metrics=['accuracy']
)

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/New Plant Diseases Dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1100 images belonging to 10 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/New Plant Diseases Dataset/test',
                                            target_size = (224, 224),
                                            batch_size =32,
                                            class_mode = 'categorical')

Found 550 images belonging to 10 classes.


In [ ]:
model.fit(training_set, validation_data = test_set, epochs =25)

Epoch 1/5
138/138 [==============================] - 20s 146ms/step - loss: 0.2959 - accuracy: 0.9091 - val_loss: 0.3302 - val_accuracy: 0.8964
Epoch 2/5
138/138 [==============================] - 20s 147ms/step - loss: 0.2980 - accuracy: 0.9045 - val_loss: 0.2307 - val_accuracy: 0.9255
Epoch 3/5
138/138 [==============================] - 24s 170ms/step - loss: 0.2855 - accuracy: 0.9045 - val_loss: 0.5351 - val_accuracy: 0.8327
Epoch 4/5
138/138 [==============================] - 21s 149ms/step - loss: 0.2607 - accuracy: 0.9191 - val_loss: 0.3377 - val_accuracy: 0.9018
Epoch 5/5
138/138 [==============================] - 20s 146ms/step - loss: 0.2434 - accuracy: 0.9200 - val_loss: 0.3057 - val_accuracy: 0.8982


In [ ]:
model.predict(test_set)

array([[9.99618530e-01, 4.21711047e-05, 1.88180150e-06, ...,
        1.68145401e-04, 2.00745678e-08, 1.02017875e-05],
       [1.35615774e-04, 9.99819458e-01, 1.27163921e-05, ...,
        1.68572740e-06, 1.75792408e-07, 1.82208792e-09],
       [1.01260111e-05, 5.30754405e-05, 1.32007199e-06, ...,
        9.99872565e-01, 1.22884812e-05, 3.19154606e-06],
       ...,
       [2.18281826e-07, 2.66760412e-06, 6.58933423e-05, ...,
        6.30091608e-06, 6.67526479e-07, 9.99507785e-01],
       [9.69940913e-04, 1.00746965e-02, 1.28521053e-02, ...,
        1.02415599e-03, 3.81607423e-03, 8.91997278e-01],
       [1.74882064e-06, 6.85480882e-06, 3.43856004e-06, ...,
        5.09629666e-04, 7.74866879e-01, 7.65123914e-05]], dtype=float32)